### Importing necessary libraries

In [ ]:
# Data manipulation
import numpy as np
import pandas as pd
import os
import random

# Data visualization
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# Image manipulation
import cv2
from PIL import Image

### Loading and visualization training and testing data

In [ ]:
train_csv = pd.read_csv("../input/gtsrb-german-traffic-sign/Train.csv")

sns.jointplot(x='Width', y='Height', data=train_csv, kind='kde')
sns.jointplot(x='Width', y='Height', data=train_csv.loc[(train_csv['Width'] < 75) & (train_csv['Height'] < 75)], kind='kde')
sns.jointplot(x='Width', y='Height', data=train_csv.loc[(train_csv['Width'] < 50) & (train_csv['Height'] < 50)], kind='kde')

plt.show()

In [ ]:
classes_number = len(os.listdir('../input/gtsrb-german-traffic-sign/Train'))
height = 32
width = 32
print("Number of traffic sign classes: ", classes_number)

In [ ]:
classes_dict = { 0:'Speed limit (20km/h)',
                 1:'Speed limit (30km/h)', 
                 2:'Speed limit (50km/h)', 
                 3:'Speed limit (60km/h)', 
                 4:'Speed limit (70km/h)', 
                 5:'Speed limit (80km/h)', 
                 6:'End of speed limit (80km/h)', 
                 7:'Speed limit (100km/h)', 
                 8:'Speed limit (120km/h)', 
                 9:'No passing', 
                 10:'No passing veh over 3.5 tons', 
                 11:'Right-of-way at intersection', 
                 12:'Priority road', 
                 13:'Yield', 
                 14:'Stop', 
                 15:'No vehicles', 
                 16:'Veh > 3.5 tons prohibited', 
                 17:'No entry', 
                 18:'General caution', 
                 19:'Dangerous curve left', 
                 20:'Dangerous curve right', 
                 21:'Double curve', 
                 22:'Bumpy road', 
                 23:'Slippery road', 
                 24:'Road narrows on the right', 
                 25:'Road work', 
                 26:'Traffic signals', 
                 27:'Pedestrians', 
                 28:'Children crossing', 
                 29:'Bicycles crossing', 
                 30:'Beware of ice/snow',
                 31:'Wild animals crossing', 
                 32:'End speed + passing limits', 
                 33:'Turn right ahead', 
                 34:'Turn left ahead', 
                 35:'Ahead only', 
                 36:'Go straight or right', 
                 37:'Go straight or left', 
                 38:'Keep right', 
                 39:'Keep left', 
                 40:'Roundabout mandatory', 
                 41:'End of no passing', 
                 42:'End no passing veh > 3.5 tons' }

In [ ]:
from matplotlib.image import imread

rows, columns = 7, 6
dir_path = "../input/gtsrb-german-traffic-sign/Train" 
classes = len(os.listdir(dir_path))
    
fig, axes = plt.subplots(rows , columns, figsize=(40, 30))
k = 0
for row in range(rows):
    for column in range(columns):
        class_path = dir_path + '/' + str(k)
        image_path = class_path + '/' + str(os.listdir(class_path)[0])
        ax = axes[row, column]
        ax.set_title(classes_dict[k], loc='center', fontsize=20)
        ax.imshow(imread(image_path))
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
        k += 1
plt.show()

In [ ]:
X_train = []
y_train = []
dir_path = "../input/gtsrb-german-traffic-sign/Train" 

for classe in range(classes): 
    class_path = dir_path + '/' + str(classe)
    for image in os.listdir(class_path):
        image_path = class_path + '/' + str(image)
        img = cv2.imread(image_path, cv2.IMREAD_COLOR)
        img = Image.fromarray(img, 'RGB')
        img = img.resize((height, width))
        X_train.append(np.array(img))
        y_train.append(classe)
        
X_train = np.array(X_train)
y_train = np.array(y_train)

In [ ]:
X_test = []

test_data = pd.read_csv("../input/gtsrb-german-traffic-sign/Test.csv")
dir_path = "../input/gtsrb-german-traffic-sign"
y_test = test_data['ClassId'].values

for image in test_data['Path'].values:
    image_path = os.path.join(dir_path, image)
    img = cv2.imread(image_path, cv2.IMREAD_COLOR)
    img = Image.fromarray(img, 'RGB')
    img = img.resize((height, width))
    X_test.append(np.array(img))

X_test = np.array(X_test)

In [ ]:
print('Training dataset shape: ', X_train.shape, y_train.shape)
print('Testing dataset shape: ',  X_test.shape, y_test.shape)

In [ ]:
rows, columns = 5, 5

sample_signs = []
random_classes = random.sample([i for i in range(classes)], rows*columns)

for i in range(rows*columns):
    index = np.random.choice(np.where(y_train == random_classes[i])[0])
    sample_signs.append(index)
    
fig, axes = plt.subplots(rows , columns, figsize=(25, 25))
k = 0
for row in range(rows):
    for column in range(columns):
        ax = axes[row, column]
        ax.set_title(classes_dict[y_train[sample_signs[k]]], loc='center', fontsize=20)
        ax.imshow(X_train[sample_signs[k]], cmap=plt.get_cmap('gray'))
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
        k += 1
plt.show()

In [ ]:
plt.style.use('seaborn-dark')

plt.figure(figsize=(14,6))
plt.title('Distribution of traffic signs images')
plt.bar(list(classes_dict.values()), 
        pd.Series(y_train, name='Traffic sign class').value_counts().sort_index().values)
plt.xticks(rotation=90)
plt.show()

In [ ]:
signs = pd.Series(y_train, name='Traffic sign class').value_counts().sort_index()
signs.rename(index = classes_dict).sort_values(ascending=False)

### Data preprocessing

In [ ]:
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

In [ ]:
from keras.utils.np_utils import to_categorical

In [ ]:
index = random.choice(range(len(y_train)))
print('Before encoding: ', y_train[index])
y_train = to_categorical(y_train, num_classes=classes)
print('After encoding: ', y_train[index])

### Splitting learning dataset into training and validation sets

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_val, Y_train, Y_val = train_test_split(X_train, y_train, random_state=42, test_size=0.25)

In [ ]:
print('Training dataset shape: ', X_train.shape, Y_train.shape)
print('Validation dataset shape: ',  X_val.shape, Y_val.shape)

### Data augmentation

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
data_gen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=False,
    vertical_flip=False,
    fill_mode="nearest")

### Building convolutional neural network

In [ ]:
from keras import layers
from keras import models

In [ ]:
model = models.Sequential()

model.add(layers.Conv2D(filters=32, kernel_size=(5, 5), activation='relu', input_shape=(height, width, 3)))
model.add(layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Dropout(rate=0.25))

model.add(layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Dropout(rate=0.25))

model.add(layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))

model.add(layers.Flatten())

model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(rate=0.25))
model.add(layers.Dense(classes, activation='softmax'))

model.summary()

In [ ]:
from tensorflow.keras.optimizers import Adam

In [ ]:
model.compile(optimizer=Adam(lr=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(data_gen.flow(X_train, Y_train, batch_size=32),
          epochs=20, 
          validation_data=(X_val, Y_val))

In [ ]:
epochs = range(1, len(history.history['accuracy']) + 1)

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

axes[0].plot(epochs, history.history['accuracy'], 'bo', label='Training set accuracy')
axes[0].plot(epochs, history.history['val_accuracy'], 'b', label='Validation set accuracy')
axes[0].set_title('Training and validation accuracy')
axes[0].set_xticks(range(2, len(history.history['accuracy']) + 1)[::2])
axes[0].set_xlabel('Number of epochs')
axes[0].set_ylabel('Accuracy')
axes[0].legend()

axes[1].plot(epochs, history.history['loss'], 'bo', label='Training set loss')
axes[1].plot(epochs, history.history['val_loss'], 'b', label='Validation set loss')
axes[1].set_title('Training and validation loss')
axes[1].set_xticks(range(2, len(history.history['accuracy']) + 1)[::2])
axes[1].set_xlabel('Number of epochs')
axes[1].set_ylabel('Loss')
axes[1].legend()

plt.show()

### Predictions and score

In [ ]:
y_pred = np.argmax(model.predict(X_test), axis=-1)

In [ ]:
bad_predictions = []
i = 0
while len(bad_predictions) < 25:
    if y_pred[i] != y_test[i]:
        bad_predictions.append(i)
    i += 1

In [ ]:
rows, columns = 5, 5

fig, axes = plt.subplots(rows , columns, figsize=(25, 25))
k = 0
for row in range(rows):
    for column in range(columns):
        ax = axes[row, column]
        ax.set_title('True class: {} \n Predicted class: {}'.format(classes_dict[y_test[bad_predictions[k]]], classes_dict[y_pred[bad_predictions[k]]]), 
                    loc='center', fontsize=12)
        ax.imshow(X_test[bad_predictions[k]])
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
        k += 1
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
print("Accuracy score on testing dataset: ", accuracy_score(y_test, y_pred))

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
cf = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(30, 30))
plt.title("Confusion matrix")
sns.heatmap(pd.DataFrame(cf, index = classes_dict,  columns = classes_dict), 
            annot=True, fmt="d", linewidths=0.1, cmap = 'Blues',
            xticklabels=classes_dict.values(), yticklabels=classes_dict.values(), cbar=False)
plt.xlabel("True class")
plt.ylabel("Predicted class")
plt.show()

In [ ]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("model.h5")